In [2]:
import pandas as pd
import numpy as np
import json


In [3]:
# Store filepath in a variable
flu_vacc_file1= "Data/NYVacc2015-16CalcData.csv"
flu_vacc_file2= "Data/NYVacc2016-17CalcData.csv"

In [4]:
vacc15_16 = pd.read_csv(flu_vacc_file1)
vacc16_17 = pd.read_csv(flu_vacc_file2)
vacc16_17

,Month,Percentage,Sample_Size
0,Jul 2016,0.0045,46338
1,Aug 2016,0.0211,46338
2,Sep 2016,0.0998,46338
3,Oct 2016,0.2633,46338
4,Nov 2016,0.3526,46338
5,Dec 2016,0.3915,46338
6,Jan 2017,0.4281,46338
7,Feb 2017,0.4466,46338
8,Mar 2017,0.4539,46338
9,Apr 2017,0.4582,46338


In [5]:
vacctotal = [vacc15_16, vacc16_17]
mergetotal = pd.concat(vacctotal)
mergetotal

,Month,Percentage,Sample_Size
0,Jul 2015,0.0053,50801
1,Aug 2015,0.0181,50801
2,Sep 2015,0.0912,50801
3,Oct 2015,0.2596,50801
4,Nov 2015,0.3578,50801
5,Dec 2015,0.3922,50801
6,Jan 2016,0.4191,50801
7,Feb 2016,0.4353,50801
8,Mar 2016,0.4468,50801
9,Apr 2016,0.4536,50801


In [7]:
mergetotal['Change']=mergetotal.Percentage.pct_change()
mergetotal.to_csv("NYVacc2015-17.csv")

In [9]:
flu_vacc_file3= "Data/NYVacc2015-17.csv"
vacc15_17 = pd.read_csv(flu_vacc_file3)
vacc15_17

,Unnamed: 0,Month,Percentage,Sample_Size,Change
0,0,Jul-15,0.0053,50801,NaN
1,1,Aug-15,0.0181,50801,2.415094
2,2,Sep-15,0.0912,50801,4.038674
3,3,Oct-15,0.2596,50801,1.846491
4,4,Nov-15,0.3578,50801,0.378274
5,5,Dec-15,0.3922,50801,0.096143
6,6,Jan-16,0.4191,50801,0.068587
7,7,Feb-16,0.4353,50801,0.038654
8,8,Mar-16,0.4468,50801,0.026419
9,9,Apr-16,0.4536,50801,0.015219
